# Downloading Reference Genomes and Whole Genome SRA (Sequence Read Archive) Data for Downstream Use in GATK (Genome Analysis Toolkit)

## 1. Preamble:

In bioinformatics, we almost always utilize genetic data from other studies on databases such as the NCBI servers. In the case of my project, I'll be utilizing entirely preexisting data of macaques in the form of whole genome data from NCBI (don't worry, this tutorial won't download any whole genome data from macaques, instead I'll use _E. coli_ for this tutorial because it's very small). The rhesus macaque is a common model organism for medical studies so we have a chromosome-level assembly for use as a reference genome (see step 2 for search and download process).

Most of the whole genome data on NCBI is found as SRA run reads. When sequences are output from a sequencing machine, they are a jumbled mess of small "short reads". In order for these short reads to be useful to us, they need to be downloaded from a database (in this case the NCBI servers), and written as `.sra` files (see step 3 for `.sra` download process). SRA files are used in combination with reference genome FASTA files in the Genome Analysis Toolkit (GATK) for genomic analyses. This notebook will walk users through downloading these files such that they can be used in GATK. This tutorial was written using Python 3.6 but should also work on 2.7 and 3.x.

## 2. Reference Genome Download Using ncbi-genome-download

To search for and download our reference genome, we will be using [**ncbi-genome-download**](https://github.com/kblin/ncbi-genome-download) which can be installed via pip by running the following in your command line:

`pip install ncbi-genome-download` 

or if you prefer conda, 

`conda install -c bioconda ncbi-genome-download`


**ncbi-genome-download** is great for finding reference genomes because of its ability to do "dry runs" which essentially runs your download request without actually downloading the genome. Below

In [1]:
import ncbi_genome_download as ngd ##importing ncbi-genome-download so we can run this code

ngd.download(dry_run="TRUE", 
             ##Using dry_run we can essentially search NCBI without downloading the file
             group="vertebrate_mammalian", 
             ##Macaques are vertebrate mammals, but another option could be to just use "all" to search all groups.
             assembly_level="chromosome", ##Here I request chromosome because I will be comparing phylogenetic patterns between chromosomes
             genus="Macaca", ##This field is named genus but you can enter a whole species name
             ##parallel=4, ##If you have multiple genomes you want to download, you can run parallel downloads using this
             ##format="FASTA" ##If you'd like to choose different options for file format, you can do that here but I'll be using the default, which is FASTA
            )
             

Considering the following 2 assemblies for download:
GCF_000364345.1	Macaca fascicularis
GCF_000772875.2	Macaca mulatta


0

If you'd like to see the options for every argument in ncbi-genome-downloader, please run the following for your desired argument:

In [2]:
ngd.config.NgdConfig.get_choices("assembly_level") ##options available for assembly_level

['all', 'complete', 'chromosome', 'scaffold', 'contig']

Visit the [**ncbi-genome-download**](https://github.com/kblin/ncbi-genome-download) repo to see all the arguments available in the program. The readme contains most of them but you can also explore the `core.py` file to see all the arguments.

This is irrelevant to my project but here I provide another example of running ncbi-genome-download, this time with E. coli, strain K-12, substrain MG1655:

In [2]:
ngd.download(dry_run="TRUE", ##I don't want to actually download anything in this tutorial so dry_run="TRUE"
             assembly_level="all", ##I just want any assembly-level
             group="bacteria", ##Searching through the bacterial group on NCBI
             taxid="511145") ##Here I use taxid because for bacteria, taxid is more specific (gives strain and substrain). If you simply use genus and "Escherichia coli", you will get many many results. 

Considering the following 13 assemblies for download:
GCF_000005845.2	Escherichia coli str. K-12 substr. MG1655
GCF_000269645.1	Escherichia coli str. K-12 substr. MG1655
GCF_000273425.1	Escherichia coli str. K-12 substr. MG1655
GCF_000364365.1	Escherichia coli str. K-12 substr. MG1655
GCF_000482265.1	Escherichia coli str. K-12 substr. MG1655
GCF_000517165.1	Escherichia coli str. K-12 substr. MG1655
GCF_000801205.1	Escherichia coli str. K-12 substr. MG1655
GCF_001308065.1	Escherichia coli str. K-12 substr. MG1655
GCF_001544635.1	Escherichia coli str. K-12 substr. MG1655
GCF_001566335.1	Escherichia coli str. K-12 substr. MG1655
GCF_002843685.1	Escherichia coli str. K-12 substr. MG1655
GCF_002966145.1	Escherichia coli str. K-12 substr. MG1655
GCF_003627195.1	Escherichia coli str. K-12 substr. MG1655


0

I want to use the rhesus macaque (_Macaca mulatta_) genome as my reference so I choose GCF_000772875.2 to download. However, for this tutorial, I'll actually download an E. coli genome from the previous line of code since it's small (1.4mb). This will give you an idea of what the default output path for ncbi-genome-download is like and what the file from NCBI will look like. To download a specific assembly accession number, use `assembly_accessions="GCF_000005845.2"` and replace the accession number with whatever yours is. If you have multiple results for your desired species, you can download any of them and they will likely work. In my case there is only one so I choose it and download it in FASTA format. When the file successfully downloads, a `0` is returned:

In [4]:
ngd.download(assembly_accessions="GCF_000005845.2", 
             file_format="fasta")

0

Congrats! You've searched for and downloaded a reference genome from NCBI. Now we'll move onto downloading unassembled whole genome reads from NCBI's SRA section. These unassembled SRAs will be aligned with the reference genome you just downloaded.

## 3. SRA File Download Using SRApy

kdmurray91 says they created [**SRApy**](https://github.com/kblin/ncbi-genome-download) because NCBI's SRA toolkit is arcane. I'm glad they did because I agree, the SRA toolkit is clunky. SRApy is well written but let's go through it anyway. SRApy is only available for pip install. Run `pip install srapy` to install srapy. Most, if not all, dependencies should automatically download. For Debian systems, run `sudo apt install libxml2-dev libxslt1-dev` if you have issues with dependencies. Now let's get started:

In [3]:
import srapy ##import SRApy

srapy.Entrez.email="_________" ##Enter your email address here before running the remainder of the code. NCBI makes you enter your email for downloading multiple files. You'll get warned if you make an unreasonable request and overload their servers.

SRA files are weird. An SRA on NCBI is actually several datatypes in one, five to be exact. What we're interested is SRX files, which are the experiment accession numbers. This will provide SRApy with what it needs to download the SRR, which is the actual sequence read data. The easiest way to find your SRX numbers is to search through [**NCBI's Bioprojects**](https://www.ncbi.nlm.nih.gov/bioproject/) for taxa you are interested in and obtain experiment (SRX) numbers. These can be dumped into a text file, separated by newlines. For the purpose of this tutorial, I've selected two small (<41mb files) E. coli SRX numbers and added them into a text file (found in the data folder of this repo, named `sraids.txt`).

We then open the text file:

In [6]:
sra_id=open("./data/sraids.txt", 'r')

And then iterate SRApy's Run Downloader over the contents of our text file (sra_id):

In [7]:
for i in sra_id:
    srapy.download_run(i)

Retrieving run info for run SRX4984104
... done.
40.591MB  26.69 MB/s [#####################################] 100% Time: 0:00:01
Retrieving run info for run SRX4984106
... done.
12.034MB  21.22 MB/s [#####################################] 100% Time: 0:00:00


Voila! You have your downloaded `.sra` files for your specified projects.






### I'm not done but I'm working on a search tool using Biopython's Entrez so that the user can just search NCBI from the notebook and choose SRX numbers to dump into a text file that SRApy can download:

In [8]:
Entrez.email = 'nilocatelli@gmail.com'
handle = Entrez.esearch(db='sra',
                        sort='relevance',
                        retmax='2000',
                        retmode='xml', 
                        term='macaca')
results = Entrez.read(handle)

results['IdList']

NameError: name 'Entrez' is not defined

In [ ]:
import srap